## Setup

In [5]:
#!pip install tensorflow

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
import numpy as np
import pandas as pd

from IPython.display import Image
from IPython.display import display

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout

from sagemaker import get_execution_role
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.transformer import Transformer
from sagemaker.image_uris import retrieve
from sagemaker.utils import name_from_base
from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow import TensorFlowModel

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tarfile
import io
import shutil
from io import StringIO, BytesIO
import joblib

from datetime import date, timedelta
from datetime import datetime
import time
from time import gmtime, strftime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


2024-02-26 21:33:42.151899: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 21:33:42.204573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 21:33:42.204613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 21:33:42.206435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 21:33:42.214101: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 21:33:42.215297: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# upload latest inference code & dependencies to s3 - RUN WHEN INFERENCE FILES CHANGED
# !aws s3 cp --recursive inference/ s3://arbit-algo/sagemaker/algo-v1/input/inference/
# !aws s3 cp --recursive code_sku/ s3://arbit-algo/sagemaker/algo-v1/input/code_sku/
# !aws s3 cp --recursive code1/ s3://arbit-algo/sagemaker/algo-v1/input/code1/
# !aws s3 cp --recursive code_sku1/ s3://arbit-algo/sagemaker/algo-v1/input/code_sku1/

upload: code_sku/.ipynb_checkpoints/preprocess-checkpoint.py to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/.ipynb_checkpoints/preprocess-checkpoint.py
upload: code_sku/requirements.txt to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/requirements.txt
upload: code_sku/__pycache__/preprocess.cpython-310.pyc to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/__pycache__/preprocess.cpython-310.pyc
upload: code_sku/util.py to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/util.py
upload: code_sku/preprocess.py to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/preprocess.py
upload: code_sku/inference.py to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/inference.py
upload: code_sku/.ipynb_checkpoints/inference-checkpoint.py to s3://arbit-algo/sagemaker/algo-v1/input/code_sku/.ipynb_checkpoints/inference-checkpoint.py
upload: code_sku1/requirements.txt to s3://arbit-algo/sagemaker/algo-v1/input/code_sku1/requirements.txt
upload: code_sku1/.ipynb_checkpoints/preprocess-checkpoi

In [25]:
# # Define IAM role
# role = get_execution_role()
# my_region = boto3.session.Session().region_name # set the region of the instance
# aws_account_id = boto3.client('sts').get_caller_identity().get('Account')
# print("RoleArn: {}".format(role))

In [4]:
# needed functions

def regression_scores(y_test, preds, transformed=False):
    '''
    Returns and prints evaluation metics for a regression model
    '''
    if transformed:

        mse = mean_squared_error(y_test, preds)
        rmse =  mean_squared_error(y_test, preds, squared=False)
        rmse_exp = mean_squared_error(np.exp(y_test), np.exp(preds), squared=False)
        mae = mean_absolute_error(y_test, preds)
        mae_exp = mean_absolute_error(np.exp(y_test), np.exp(preds))

    else:
        mse = mean_squared_error(y_test, preds)
        rmse =  mean_squared_error(y_test, preds, squared=False)
        rmse_exp = 'N/A'
        mae = mean_absolute_error(y_test, preds)
        mae_exp = 'N/A'
        
    r2 = r2_score(y_test, preds)
    adj_r2 = 'N/A'

    print('MSE: ', mse)
    print('RMSE: ', rmse)
    print('RMSE (retuned to normal scale): ', rmse_exp)
    print('MAE: ', mae)
    print('MAE (retuned to normal scale): ', mae_exp)
    print('R-squared: ', r2)

    return mse, rmse, rmse_exp, mae, mae_exp, r2, adj_r2

def score_table(scores, model_name, y_test, preds, transformed=False, notes=None):
    '''
    creates a data frame with various scores for each model
    '''
    
    mse, rmse, rmse_exp, mae, mae_exp, r2, adj_r2 = regression_scores(y_test, preds, transformed)
    
    score_list = []
    score_list.extend((mse, rmse, rmse_exp, mae, mae_exp, r2, adj_r2, notes))
    
    scores.loc[model_name] = score_list
    return scores

In [28]:
!aws s3 cp --recursive s3://arbit-algo/sagemaker/algo-v1/input/code_sku/ code_sku/
!aws s3 cp --recursive s3://arbit-algo/sagemaker/algo-v1/input/code_sku1/ code_sku1/

download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/.ipynb_checkpoints/inference-checkpoint.py to code_sku/.ipynb_checkpoints/inference-checkpoint.py
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/requirements.txt to code_sku/requirements.txt
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/__pycache__/preprocess.cpython-310.pyc to code_sku/__pycache__/preprocess.cpython-310.pyc
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/.ipynb_checkpoints/preprocess-checkpoint.py to code_sku/.ipynb_checkpoints/preprocess-checkpoint.py
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/inference.py to code_sku/inference.py
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/util.py to code_sku/util.py
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku/preprocess.py to code_sku/preprocess.py
download: s3://arbit-algo/sagemaker/algo-v1/input/code_sku1/inference.py to code_sku1/inference.py
download: s3://arbit-algo/sagemaker/algo-v1/inpu

## Inference

### Model 3

In [5]:
get_last_modified = lambda obj: int(obj['LastModified'].strftime('%s'))
s3 = boto3.client('s3')
objs = s3.list_objects_v2(Bucket='arbit-algo', Prefix='sagemaker/algo-v1/output/models/3/')['Contents']
keys_with_model_tar_gz = [item for item in objs if 'model.tar.gz' in item['Key']]
last_added = [obj['Key'] for obj in sorted(keys_with_model_tar_gz, key=get_last_modified, reverse=True)][0]

if os.path.exists('1') and os.path.isdir('1'):
    shutil.rmtree('1')

s3_object = s3.get_object(Bucket='arbit-algo', Key=last_added)

wholefile = s3_object['Body'].read()
fileobj = io.BytesIO(wholefile)
tarf = tarfile.open(fileobj=fileobj)
names = tarf.getnames()
for name in names:
    print(name)

model_files = [names]
tarf.extractall()

model = tf.keras.models.load_model('1')

1
1/keras_metadata.pb
1/assets
1/saved_model.pb
1/variables
1/variables/variables.data-00000-of-00001
1/variables/variables.index


In [6]:
## SKU
from code_sku.preprocess import *

# PROCESSING DATA
model_data = 's3://arbit-algo/sagemaker/algo-v1/processing/input/inference_sku/input_sku.csv'
training_data = 's3://arbit-algo/sagemaker/algo-v1/processing/input/input_sku_training.csv'

# just to make sure we capture all columns
X_train_preprocessed3 = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/X_train_sku.csv').iloc[:, 1:]

df_model = final_sku(training_data) #from preprocess_v1
df_model = df_model[(df_model['SOLD_PRICE'] > 5) & (df_model['RETAILPRICE']>5) & (df_model['num_sales']>=1)]
df_model = df_model.dropna().reset_index(drop=True)

X, y = final_preprocess_sku(df_model)
#product_info_agg = X.to_csv('product_info.csv', index=True)

# TODO: decide - should I do this on train data only??
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio, random_state=10)

bool_cols = ['IS_COLLAB', 'IS_OG_SE']
cat = ['GENDER', 'SILHOUETTE', 'COLOR', 'BRAND', 'primary_source']
cont = ['RETAILPRICE', 'num_sales', 'std_sale_px', 'DAYS_SINCE_LAST_SALE', 'LOWEST_ASK', #'avg_sale_px', 
        'DAYS_SINCE_RELEASE', 'source_count', 'lag0', 'lag1', 'lag2', 'lag3', 'lag4']

scaler = StandardScaler().fit(X[cont]) # or X_train
ohe = OneHotEncoder(handle_unknown='ignore').fit(X[cat])

# TODO: for batch don't need to load in chunks - can edit this in preprocess file
df_new = final_sku(model_data, inference=True)
df_new = string_cleanup(df_new)
df_new = df_new.dropna().reset_index(drop=True) # drop any final missing values
df_new.set_index('SKU', inplace=True)

X_new, y_new = final_preprocess_sku(df_new)

# drop any final missing & extreme values
# deleting any infinity values (already scaled and logged, so this shouldnt be dropping any actual columns)

print('length of X_new:', len(X_new))
#X_new = X_new.reset_index(drop=True)
mask = X_new.isna()
X_new = X_new[~mask]#.reset_index(drop=True)

# drop corresponing rows in model_data
print('length of df_new:', len(df_new))

df_new = df_new[~mask.any(axis=1)]#.reset_index(drop=True)
print('lengths match?', len(X_new)==len(df_new))

#df_new.to_csv('s3://arbit-algo/sagemaker/algo-v1/processing/input/input_transformed.csv', index=False)

# one hot encode, scale & concat
df_ohe_new = pd.DataFrame(ohe.transform(X_new[cat]).toarray(), 
                          columns=ohe.get_feature_names(X_new[cat].columns), index=X_new[cat].index)
X_new_cont = pd.DataFrame(scaler.transform(X_new[cont]),columns=X_new[cont].columns,index=X_new[cont].index)
bool_cols_new = pd.get_dummies(X_new[bool_cols], columns=bool_cols, drop_first= True)

X_new_preprocessed = pd.concat([X_new_cont, df_ohe_new], axis=1)
X_new_preprocessed = pd.concat([X_new_preprocessed, bool_cols_new], axis=1)

#X_new = scaler.transform(X_new)

# prefix = "contains_"
# X_new_preprocessed = X_new_preprocessed.join(X_new.filter(regex=f'^{prefix}'))

# Get missing columns inthe training test
missing_cols = set(X_train_preprocessed3.columns) - set(X_new_preprocessed.columns)
for c in missing_cols:
    X_new_preprocessed[c] = 0
X_new_preprocessed = X_new_preprocessed[X_train_preprocessed3.columns]
#X_new_preprocessed = X_new_preprocessed.iloc[:,1:]

# uncomment if needed
#X_new_preprocessed.to_csv('X_inference_sku.csv', index=False) 

/root/historical-sales-analysis/code_sku/preprocess.py:90: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunks in pd.read_csv(file_path, chunksize=100000, header=0):
/root/historical-sales-analysis/code_sku/preprocess.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RELEASEDATE'][df.RELEASEDATE.isna()] = df['RELEASEDATE'][~df.RELEASEDATE.isna()].quantile(0.5, interpolation="midpoint")


AttributeError: Can only use .dt accessor with datetimelike values

In [43]:
preds = model.predict(X_new_preprocessed)
df = pd.DataFrame(np.exp(preds), columns=['predictions'])

product_info = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/output/product_info_sku.csv') # this needs to be non standardized
# product_info = pd.read_csv('input_transformed.csv').reset_index(drop=True)
# product_info = product_info.merge(size_map, on=['SKU', 'SIZE', 'GENDER'], how='left')

df_merge = pd.concat([df, product_info], axis=1)

mean_std_pct_non_zero = np.mean(df_merge['std_sale_px'][df_merge.std_sale_px>0] / df_merge['predictions'][df_merge.std_sale_px>0])

# if std deviation is 0, replace with mean of std dev as pct of prediction value for all non zero std dev
df_merge['std_sale_px'][df_merge.std_sale_px==0] = df_merge.predictions * mean_std_pct_non_zero

# if less than 5 sales and std dev as % of pred is less than mean, replace with mean
df_merge['std_sale_px'][(df_merge.std_sale_px / df_merge.predictions < mean_std_pct_non_zero) & (df_merge.num_sales < 10)] = \
    df_merge.predictions * mean_std_pct_non_zero

# using model_training errors
model_info = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/output/evaluation.csv')
mae = model_info['mae_exp'][(~model_info.notes.isna()) & (model_info['notes'].str.contains('Model 3'))].iloc[-1]
# mae_individual = mae * (df_merge.predictions / np.mean(df_merge.predictions))
mae_individual = mae * (df_merge.std_sale_px / np.mean(df_merge.std_sale_px)) # using sneaker's standard deviation of price

# need to get original unchanged size
df_final = df_merge[['SKU', 'GENDER', 'RELEASEDATE', 'predictions']] #'GENDER',
                     #'num_sales', 'DAYS_SINCE_LAST_SALE', 'DAYS_SINCE_RELEASE', 'std_sale_px', 'avg_sale_px_last_5', 'lag1', 'lag2', 'lag3']] # #'avg_sale_px', 
df_final['prediction_low'] = df_final['predictions'] - mae_individual
df_final['prediction_low'] = df_final['prediction_low'].apply(lambda x: max(0, x))

df_final['prediction_high'] = df_final['predictions'] + mae_individual
df_final['SKU'] = df_final['SKU'].str.upper()

df_final['predictions'] = df_final['predictions'].apply(lambda x: "%.2f" % x)
df_final['prediction_low'] = df_final['prediction_low'].apply(lambda x: "%.2f" % x)
df_final['prediction_high'] = df_final['prediction_high'].apply(lambda x: "%.2f" % x)

from datetime import date, timedelta
from datetime import datetime
import time
from time import gmtime, strftime

df_final['pred_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#df_final['pred_date'] = '2023-10-22'

# save today's prediction - UNCOMMENT
#df_final.to_csv('preds-9-26.csv')
#df_final.to_csv(f's3://arbit-algo/sagemaker/algo-v1/output/predictions/dt-{date.today()}/predictions.csv')

# save historical prediction TODO: change mode to 'a'
#output_path = 's3://arbit-algo/sagemaker/algo-v1/output/pred_history.csv'
#df_final.to_csv(output_path, mode='a', header=False, index=False) #not os.path.exists(output_path))

df_final.to_csv(f's3://justin-automation-output/outputs/output/predictions-sku/dt-{date.today()}/predictions_sku.csv')
#df_final.to_csv(f's3://arbit-algo/sagemaker/algo-v1/output/predictions-sku/dt-2023-10-22/predictions_sku.csv')

print('Model 3 Preds:')
display(df_final.head())

Model 3 Preds:


<ipython-input-43-b1a87433c415>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['std_sale_px'][df_merge.std_sale_px==0] = df_merge.predictions * mean_std_pct_non_zero
<ipython-input-43-b1a87433c415>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['std_sale_px'][(df_merge.std_sale_px / df_merge.predictions < mean_std_pct_non_zero) & (df_merge.num_sales < 10)] = \
<ipython-input-43-b1a87433c415>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,SKU,GENDER,RELEASEDATE,predictions,prediction_low,prediction_high,pred_date
0,019000,men,2020-12-08,81.398308,66.372344,96.424271,2023-12-21 21:47:20
1,019099,men,2020-06-29,70.387589,64.975297,75.799880,2023-12-21 21:47:20
2,034563,men,2014-09-16,93.974236,88.003323,99.945149,2023-12-21 21:47:20
3,036516,child,2016-05-12,69.727715,62.441082,77.014347,2023-12-21 21:47:20
4,100000317,men,2021-02-12,71.493988,62.879719,80.108257,2023-12-21 21:47:20


### Model 4

In [45]:
get_last_modified = lambda obj: int(obj['LastModified'].strftime('%s'))
s3 = boto3.client('s3')
objs = s3.list_objects_v2(Bucket='arbit-algo', Prefix='sagemaker/algo-v1/output/models/4/')['Contents']
keys_with_model_tar_gz = [item for item in objs if 'model.tar.gz' in item['Key']]
last_added = [obj['Key'] for obj in sorted(keys_with_model_tar_gz, key=get_last_modified, reverse=True)][0]

if os.path.exists('1') and os.path.isdir('1'):
    shutil.rmtree('1')

s3_object = s3.get_object(Bucket='arbit-algo', Key=last_added)

wholefile = s3_object['Body'].read()
fileobj = io.BytesIO(wholefile)
tarf = tarfile.open(fileobj=fileobj)
names = tarf.getnames()
for name in names:
    print(name)

model_files = [names]
tarf.extractall()

model = tf.keras.models.load_model('1')

1
1/saved_model.pb
1/variables
1/variables/variables.index
1/variables/variables.data-00000-of-00001
1/assets
1/keras_metadata.pb


In [46]:
from code_sku1.preprocess import *

model_data = 's3://arbit-algo/sagemaker/algo-v1/processing/input/inference_sku/input_sku.csv'
training_data = 's3://arbit-algo/sagemaker/algo-v1/processing/input/input_sku_training.csv'

# just to make sure we capture all columns
X_train_preprocessed4 = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/train/X_train_sku2.csv')

df_model = final_sku(training_data) #from preprocess_v1
df_model = df_model[(df_model['SOLD_PRICE'] > 5) & (df_model['RETAILPRICE']>5) & (df_model['num_sales']>=1)]
df_model = df_model.dropna().reset_index(drop=True)

X, y = final_preprocess_sku(df_model)
#product_info_agg = X.to_csv('product_info.csv', index=True)

# TODO: decide - should I do this on train data only??
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio, random_state=10)

bool_cols = ['IS_COLLAB', 'IS_OG_SE']
cat = ['GENDER', 'SILHOUETTE', 'COLOR', 'BRAND', 'primary_source']
cont = ['RETAILPRICE', 'num_sales', 'std_sale_px', 'DAYS_SINCE_LAST_SALE', #'avg_sale_px', 
        'DAYS_SINCE_RELEASE', 'source_count', 'lag0', 'LOWEST_ASK']

scaler = StandardScaler().fit(X[cont]) # or X_train
ohe = OneHotEncoder(handle_unknown='ignore').fit(X[cat])

# TODO: for batch don't need to load in chunks - can edit this in preprocess file
df_new = final_sku(model_data, inference=True)
df_new = string_cleanup(df_new)
df_new = df_new.dropna().reset_index(drop=True) # drop any final missing values
df_new.set_index('SKU', inplace=True)

X_new, y_new = final_preprocess_sku(df_new)

# drop any final missing & extreme values
# deleting any infinity values (already scaled and logged, so this shouldnt be dropping any actual columns)

print('length of X_new:', len(X_new))
#X_new = X_new.reset_index(drop=True)
mask = X_new.isna()
X_new = X_new[~mask]#.reset_index(drop=True)

# drop corresponing rows in model_data
print('length of df_new:', len(df_new))

df_new = df_new[~mask.any(axis=1)]#.reset_index(drop=True)
print('lengths match?', len(X_new)==len(df_new))

#df_new.to_csv('s3://arbit-algo/sagemaker/algo-v1/processing/input/input_transformed.csv', index=False)

# one hot encode, scale & concat
df_ohe_new = pd.DataFrame(ohe.transform(X_new[cat]).toarray(), 
                          columns=ohe.get_feature_names(X_new[cat].columns), index=X_new[cat].index)
X_new_cont = pd.DataFrame(scaler.transform(X_new[cont]),columns=X_new[cont].columns,index=X_new[cont].index)
bool_cols_new = pd.get_dummies(X_new[bool_cols], columns=bool_cols, drop_first= True)

X_new_preprocessed = pd.concat([X_new_cont, df_ohe_new], axis=1)
X_new_preprocessed = pd.concat([X_new_preprocessed, bool_cols_new], axis=1)

#X_new = scaler.transform(X_new)

# prefix = "contains_"
# X_new_preprocessed = X_new_preprocessed.join(X_new.filter(regex=f'^{prefix}'))


# X_new_preprocessed_index = X_new_preprocessed.index

# X_new_preprocessed = \
# X_new_preprocessed.reset_index(drop=True).join(X_new.reset_index(drop=True).filter(regex=f'^{prefix}'))

# X_new_preprocessed.index = X_new_preprocessed_index


# Get missing columns in the training test
missing_cols = set(X_train_preprocessed4.columns) - set(X_new_preprocessed.columns)
for c in missing_cols:
    X_new_preprocessed[c] = 0
X_new_preprocessed = X_new_preprocessed[X_train_preprocessed4.columns]
X_new_preprocessed = X_new_preprocessed.iloc[:,1:]

X_new_preprocessed
#X_new_preprocessed.to_csv('s3://arbit-algo/sagemaker/algo-v1/output/X_inference_sku2.csv', index=False) 

/root/historical-sales-analysis/code_sku1/preprocess.py:404: DtypeWarning: Columns (9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  data = final_cleanup_sku(data_filepath, min_sale, inference=inference)
/root/historical-sales-analysis/code_sku1/preprocess.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RELEASEDATE'][df.RELEASEDATE.isna()] = df['RELEASEDATE'][~df.RELEASEDATE.isna()].quantile(0.5, interpolation="midpoint")
/root/historical-sales-analysis/code_sku1/preprocess.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

length of X_new: 11148
length of df_new: 11148
lengths match? True


,RETAILPRICE,DAYS_SINCE_RELEASE,source_count,num_sales,std_sale_px,DAYS_SINCE_LAST_SALE,lag0,LOWEST_ASK,GENDER_child,GENDER_infant,...,BRAND_saucony,BRAND_under armour,BRAND_vans,BRAND_veja,primary_source_alias,primary_source_ebay,primary_source_grailed,primary_source_stockx,IS_COLLAB_true,IS_OG_SE_true
SKU,,,,,,,,,,,,,,,,,,,,,
019000,-1.392345,1.432047,2.082944,1.301146,-0.270047,-0.459698,-0.684657,0.695041,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
019099,-0.915112,1.849686,-0.430751,0.392275,-1.576516,5.462431,-1.288386,-0.602583,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
034563,-0.915112,7.297045,-0.430751,0.392275,-1.450838,0.395720,-0.800659,0.483800,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
036516,-1.976431,5.739920,-0.430751,0.128679,-1.196043,1.316940,-1.383288,-1.188048,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
100000317,-0.915112,1.261897,-0.430751,-0.610374,-0.981889,0.461522,-1.699993,-0.386190,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wtntrck4,-1.572936,-0.210153,-1.687599,-1.297945,-1.561254,0.922132,-1.699993,-0.738479,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
wtntrcn4,-1.572936,-0.210153,-1.687599,-0.986040,-1.096436,-0.196493,-1.517621,-0.084273,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
ww577vk,-1.064726,-0.772162,-1.687599,-0.628058,-1.604738,1.119536,-1.351128,0.077072,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0


In [47]:
preds = model.predict(X_new_preprocessed)
df = pd.DataFrame(np.exp(preds), columns=['predictions'])

product_info = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/output/product_info_sku1.csv') # this needs to be non standardized
# product_info = pd.read_csv('input_transformed.csv').reset_index(drop=True)
# product_info = product_info.merge(size_map, on=['SKU', 'SIZE', 'GENDER'], how='left')

df_merge = pd.concat([df, product_info], axis=1)

## ASSIGNED EARLIER
# mean_std_pct_non_zero = np.mean(df_merge['std_sale_px'][df_merge.std_sale_px>0] / df_merge['predictions'][df_merge.std_sale_px>0])

# if std deviation is 0, replace with mean of std dev as pct of prediction value for all non zero std dev
df_merge['std_sale_px'][df_merge.std_sale_px==0] = df_merge.predictions * mean_std_pct_non_zero

# if less than 5 sales and std dev as % of pred is less than mean, replace with mean
df_merge['std_sale_px'][(df_merge.std_sale_px / df_merge.predictions < mean_std_pct_non_zero) & (df_merge.num_sales < 5)] = \
    df_merge.predictions * mean_std_pct_non_zero

# using model_training errors
model_info = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/output/evaluation.csv')
mae = model_info['mae_exp'][(~model_info.notes.isna()) & (model_info['notes'].str.contains('Model 4'))].iloc[-1]
# mae_individual = mae * (df_merge.predictions / np.mean(df_merge.predictions))
mae_individual = mae * (df_merge.std_sale_px / np.mean(df_merge.std_sale_px)) # using sneaker's standard deviation of price

# need to get original unchanged size
df_final2 = df_merge[['SKU', 'GENDER', 'RELEASEDATE', 'predictions']] #'GENDER',
                     #'num_sales', 'DAYS_SINCE_LAST_SALE', 'DAYS_SINCE_RELEASE', 'std_sale_px', 'avg_sale_px_last_5', 'lag1', 'lag2', 'lag3']] # #'avg_sale_px', 
df_final2['prediction_low'] = df_final2['predictions'] - mae_individual
df_final2['prediction_low'] = df_final2['prediction_low'].apply(lambda x: max(0, x))

df_final2['prediction_high'] = df_final2['predictions'] + mae_individual
df_final2['SKU'] = df_final2['SKU'].str.upper()

# round to 2 decimals
df_final2['predictions'] = df_final2['predictions'].apply(lambda x: "%.2f" % x)
df_final2['prediction_low'] = df_final2['prediction_low'].apply(lambda x: "%.2f" % x)
df_final2['prediction_high'] = df_final2['prediction_high'].apply(lambda x: "%.2f" % x)


from datetime import date, timedelta
from datetime import datetime
import time
from time import gmtime, strftime

df_final2['pred_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#df_final2['pred_date'] = '2023-10-22'

#df_final = pd.read_csv(f's3://arbit-algo/sagemaker/algo-v1/output/predictions-sku/dt-2023-10-22/predictions_sku.csv')

df_final2 = df_final2[~df_final2.SKU.isin(list(set(df_final.SKU)))].reset_index(drop=True)

df_final2.to_csv(f's3://justin-automation-output/outputs/output/predictions-sku/dt-{date.today()}/predictions_sku2.csv')
#df_final2.to_csv(f's3://arbit-algo/sagemaker/algo-v1/output/predictions-sku/dt-2023-10-22/predictions_sku2.csv')

print('Model 4 Preds:')
display(df_final2.head())

Model 4 Preds:


<ipython-input-47-dfd46994fd55>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['std_sale_px'][df_merge.std_sale_px==0] = df_merge.predictions * mean_std_pct_non_zero
<ipython-input-47-dfd46994fd55>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['std_sale_px'][(df_merge.std_sale_px / df_merge.predictions < mean_std_pct_non_zero) & (df_merge.num_sales < 5)] = \
<ipython-input-47-dfd46994fd55>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,SKU,GENDER,RELEASEDATE,predictions,prediction_low,prediction_high,pred_date
0,101,men,2021-04-26,101.567223,53.366824,149.767621,2023-12-21 21:48:04
1,1011A824-026,men,2021-10-07,73.168762,57.580430,88.757094,2023-12-21 21:48:04
2,1011A824-402,men,2022-03-10,50.501717,44.950511,56.052922,2023-12-21 21:48:04
3,1011A826-021,men,2022-06-03,53.291161,51.819175,54.763146,2023-12-21 21:48:04
4,1011A993-020,men,2021-08-21,53.503006,49.964793,57.041219,2023-12-21 21:48:04


## Monitoring

In [35]:
# evaluate model
scores = pd.DataFrame(columns = ['mse', 'rmse', 'rmse_exp', 'mae', 'mae_exp', 'r2', 'adj_r2', 'notes']) 
yesterday = date.today() - timedelta(days=1)

# preds for T-1
preds1 = pd.read_csv(f's3://arbit-algo/sagemaker/algo-v1/output/predictions-sku/dt-{yesterday}/predictions_sku.csv')
preds2 = pd.read_csv(f's3://arbit-algo/sagemaker/algo-v1/output/predictions-sku/dt-{yesterday}/predictions_sku2.csv')

preds = pd.concat([preds1, preds2])
# preds = pd.concat([df_final, df_final2])

# ground truth
# X_test = pd.read_csv('s3://arbit-algo/sagemaker/algo-v1/processing/output/test/X_test.csv')
y_truth = pd.read_csv('s3://historicaldata-sample/ground_truth_sku.csv')

# dropping missing values
y_truth = y_truth.dropna()

# subset where ground truth exists, log transform to get to scale of model
final = pd.merge(y_truth[['SKU', 'RELEASEDATE', 'PRICE']], preds[['SKU', 'RELEASEDATE', 'predictions', 'prediction_low', 'prediction_high']], 
                 on=['SKU', 'RELEASEDATE'], how='inner')
final['predictions'] = np.log(final.predictions)
final['prediction_low'] = np.log(final.prediction_low)
final['prediction_high'] = np.log(final.prediction_high)
final['PRICE'] = np.log(final.PRICE)
final['price_in_range'] = final.apply(lambda x: 1 if x['prediction_low'] <= x['PRICE'] <= x['prediction_high'] else 0, axis=1)
final['eval_date'] = date.today()
final = final.drop_duplicates().reset_index(drop=True)
# final.to_csv('s3://arbit-algo/sagemaker/algo-v1/output/eval_values.csv', mode='a', header=False, index=False)

# get and save scores
model='AlgoV1'
print("Yesterday's Predictions vs Yesterday's Actual Prices, SKU Model, Evaluation: ") 
score_table(scores, model, final['PRICE'], final['predictions'], transformed=True, 
            notes=f'Monitor SKU {str(date.today())}')
eval_path = 's3://justin-automation-output/outputs/output/evaluation.csv'
scores.reset_index().to_csv(eval_path, mode='a', header=False, index=False) #(not os.path.exists(eval_path)))

Yesterday's Predictions vs Yesterday's Actual Prices, SKU Model, Evaluation: 
MSE:  0.03483704005468016
RMSE:  0.18664683242605581
RMSE (retuned to normal scale):  37.17596294567497
MAE:  0.13435643219091037
MAE (retuned to normal scale):  20.911212927183374
R-squared:  0.8522180669157295
